In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [1]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


In [2]:
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [3]:
from transformers import FlaxAutoModelForCausalLM, AutoTokenizer
import jax

/home/jet/.local/share/virtualenvs/JAX-in-Action-2DrGcPDg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import FlaxGPTJForCausalLM #?

In [7]:
#model_name = "EleutherAI/gpt-j-6B"
model_name = "EleutherAI/gpt-neo-125M"
#model_name = "EleutherAI/gpt-neo-1.3B"
#model_name = "EleutherAI/gpt-neo-2.7B"

In [8]:
model = FlaxAutoModelForCausalLM.from_pretrained(
    model_name,
#    revision="float16",
    dtype=jax.numpy.float16,
    from_pt=True
)

OSError: float16 is not a valid git identifier (branch name, tag name or commit id) that exists for this model name. Check the model page at 'https://huggingface.co/EleutherAI/gpt-neo-125M' for available revisions.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
!nvidia-smi --query-gpu=utilization.memory,memory.total,memory.free,memory.used --format=csv

In [ ]:
!nvidia-smi

In [ ]:
# https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.FlaxPreTrainedModel.to_fp16
# or bf16
model.params = model.to_fp16(model.params)

In [ ]:
!nvidia-smi --query-gpu=utilization.memory,memory.total,memory.free,memory.used --format=csv

In [ ]:
model

In [ ]:
tokenizer

In [ ]:
prompt = """Generate SQL query from the text inside triple backticks
```select all the users from table 'users' older than 20 years```
"""

In [ ]:
inputs = tokenizer(prompt, return_tensors="jax")

In [ ]:
inputs

https://huggingface.co/blog/how-to-generate

https://huggingface.co/docs/transformers/main_classes/text_generation#generation

https://huggingface.co/docs/transformers/generation_strategies



In [ ]:
generated_ids = model.generate(
    **inputs,
    do_sample=True,
    num_beams=1,
    max_new_tokens=300,
    temperature=0.7,
    pad_token_id = model.config.eos_token_id,
    prng_key=jax.random.PRNGKey(4232),
    no_repeat_ngram_size=2)

In [ ]:
generated_text = tokenizer.decode(generated_ids['sequences'].squeeze(0))
print(generated_text)